In [1]:
# !pip install ipynb

In [2]:
import os
import cadquery as cq
from cadquery import exporters
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show)
from jupyter_cadquery import set_sidecar, set_defaults

from FinalProject_ModelListGeneration import get_models_list
from tqdm import tqdm
from tqdm import trange

Overwriting auto display for cadquery Workplane and Shape


In [3]:
# set_defaults(axes=True, grid=True, axes0=True, ortho=True, transparent=False)
# set_sidecar("CadQuery", init=True)

In [4]:
'''
1. 'sections' should be dict, allowable values = 'line', 'arc90', 'arc45'
2. 'revolves' should be dict, allowable values = 'left', 'right', 'top', 'down',
   for 'line' shape the revolve value is not take into account, but should be specified in any case,
   'left' is the revolving in the direction of the '+X-axis',
   'right' is '-X-axis', 'top' is '+Y-axis', 'down' is '-Y-axis'
3. 'radiuses' should be dict, for 'line' shape the radius value is used as the length
'''

def get_geom(d = 10,
             sections=['line'],
             radiuses=[100],
             revolves=['left']):
    
    shape_param = {'line' :{'angle':0},
                   'arc90':{'angle':90},
                   'arc45':{'angle':45}}
    
    # setting the initial workplane
    result = cq.Workplane('XY')
    
    # dictionaries for collecting the amount of rotations and translations of sections
    rotate_param = []
    translate_param = []
    
    for i, shape in enumerate(sections):
        
        # setting the axis for revolving
        revolve_param = {'left' :{'startAxis':(radiuses[i], 0),
                                  'endAxis'  :(radiuses[i], 1)},
                         'right':{'startAxis':(-radiuses[i], 0),
                                  'endAxis'  :(-radiuses[i], -1)},
                         'top'  :{'startAxis':(0, radiuses[i]),
                                  'endAxis'  :(-1, radiuses[i])},
                         'down' :{'startAxis':(0, -radiuses[i]),
                                  'endAxis'  :(1, -radiuses[i])}}
        
        if shape == 'line':
            
            # extruding
            section = (cq.Workplane('XY')
                       .circle(d/2)
                       .extrude(radiuses[i]))
            
            # moving to the target place
            if i != 0:
                for j in range(i, 0, -1):
                    section = (section
                               .rotate(rotate_param[j-1]['startAxis'],
                                       rotate_param[j-1]['endAxis'],
                                       rotate_param[j-1]['angle'])
                               .translate(translate_param[j-1]))
                else:
                    pass
            
            # concatenating with result
            result = result.union(section, glue=True)
#             result.add(section)
            
            # collecting the values of rotations and translations
            translate_param.append((0,0,radiuses[i]))
            rotate_param.append({'startAxis':(25, 0),
                                 'endAxis':(25, 1),
                                 'angle':360})
            
        elif shape == 'arc90' or shape == 'arc45':
            
            # extruding (revolving)
            section = (cq.Workplane('XY')
                       .circle(d/2)
                       .revolve(shape_param[shape]['angle'],
                                revolve_param[revolves[i]]['startAxis'],
                                revolve_param[revolves[i]]['endAxis']))
            
            # moving to the target place
            if i != 0:
                for j in range(i, 0, -1):
                    section = (section
                               .rotate(rotate_param[j-1]['startAxis'],
                                       rotate_param[j-1]['endAxis'],
                                       rotate_param[j-1]['angle'])
                               .translate(translate_param[j-1]))
                else:
                    pass
            
            # concatenating with result
            result = result.union(section, glue=True)
#             result.add(section)
            
            # collecting the values of rotation and translation
            translate_param.append((0,0,0))
            rotate_param.append({'startAxis':revolve_param[revolves[i]]['startAxis'],
                                 'endAxis':revolve_param[revolves[i]]['endAxis'],
                                 'angle':shape_param[shape]['angle']})
        
        else:
            break

    return result

In [5]:
models = get_models_list()

In [6]:
for model in trange(len(models)):
    result_geom = get_geom(d = models.iloc[model]['diameter'].item(),
                           sections = [str(x) for x in list(models.iloc[model][4:7])],
                           radiuses = [x.item() for x in list(models.iloc[model][1:4])],
                           revolves = [str(x) for x in list(models.iloc[model][7:10])])

    model_name = models.iloc[model]['description'] + '.step'

    exporters.export(result_geom,
                     os.path.join('data/cad_models', model_name),
                     tolerance=0.001,
                     angularTolerance=0.05)

100%|██████████| 4116/4116 [03:15<00:00, 21.06it/s]


---

In [7]:
# %%time
# result_geom = get_geom(d = 10,
#                        sections=['arc45', 'arc45', 'arc90', 'arc90'],
#                        radiuses=[25, 25, 25, 25],
#                        revolves=['left', 'top', 'down', 'down'])

# show(result_geom, axes=True, grid=True, axes0=True, ortho=True, transparent=False)

# exporters.export(result_geom,
#                  os.path.join('data/cad_models', 'model_name.step'),
#                  tolerance=0.001,
#                  angularTolerance=0.05)

In [8]:
# def get_section_geom(d = 10, shape='line', radius=100, revolve='left'):
    
#     shape_param = {'line': {'angle':0},
#                    'arc90':{'angle':90},
#                    'arc45':{'angle':45}}
    
#     revolve_param = {'left': {'startAxis':(radius, 0),
#                               'endAxis':  (radius, 1)},
#                      'right':{'startAxis':(-radius, 0),
#                               'endAxis':  (-radius, -1)},
#                      'top':  {'startAxis':(0, radius),
#                               'endAxis':  (-1, radius)},
#                      'down': {'startAxis':(0, -radius),
#                               'endAxis':  (1, -radius)}}

#     if shape == 'line':

#         section = (cq.Workplane('XY')
#                    .circle(d/2)
#                    .extrude(radius))

#         translate_param = (0,0,radius)
#         rotate_param = {'startAxis':(25, 0),
#                         'endAxis':(25, 1),
#                         'angle':360}

#     elif shape == 'arc90' or shape == 'arc45':

#         # extruding
#         section = (cq.Workplane('XY')
#                    .circle(d/2)
#                    .revolve(shape_param[shape]['angle'],
#                             revolve_param[revolve]['startAxis'],
#                             revolve_param[revolve]['endAxis']))

#         translate_param = (0,0,0)
#         rotate_param = {'startAxis':revolve_param[revolve]['startAxis'],
#                         'endAxis':revolve_param[revolve]['endAxis'],
#                         'angle':shape_param[shape]['angle']}

#     return section, translate_param, rotate_param